In [42]:
import numpy as np
from itertools import product

class Map(object):
    '''Generates a noise field defined over a grid structure (nrows x ncols),
    where a qubit resides at every node (including edges).
    '''

    def __init__(self, nrows=1, ncols=1, m_type=0, m_vals=None):
       
        if m_vals.all() == None:
            self.m_vals = np.zeros([nrows, ncols])
        else: 
            self.m_vals = m_vals
            nrows, ncols = np.shape(m_vals)[0], np.shape(m_vals)[1]
        
        self.m_nodes_coords = list(product(range(nrows), range(ncols)))
        self.m_nodes = len(self.m_nodes_coords)
        self.m_type = 'Pauli_z_noise'
        self.m_knn = []

    def m_knn_list(self, corr_r):
        '''Updates neighest neighbours list given a correlation radius
        that sets the maximally far away neighbour.
        '''
        self.m_knn = 0 #some results

    def m_vectorise_map(self):
        '''Returns a vector formed by stacking rows of self.m_vals.
        '''
        vectormap = self.m_vals.reshape(self.m_nodes)
        return vectormap

In [43]:
testmap = np.arange(12).reshape(3,4)

In [44]:
testint = Map(m_vals=testmap)

In [45]:
testint.m_vectorise_map()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [ ]:
itertools?

In [ ]:
product?

In [ ]:
iterobject = product(range(5), range(5))

In [ ]:
next(iterobject)

In [ ]:
test = np.arange(25).reshape(5,5)
print(test)
stacked = test.reshape(25)
print(stacked)

In [ ]:
np.vstack?

In [ ]:
nrows=10
ncols=2

In [ ]:
coord = list(product(range(3), range(3)))

In [ ]:
map_val = np.zeros([nrows,ncols])
map_val

In [ ]:
r = np.arange(nrows)
c = np.arange(ncols)

def function(i, j):
    return (i,j)

In [ ]:
obj= map(function, r,c)

In [ ]:
obj

In [ ]:
for item in np.meshgrid(r,c):
    print item
    print type(item)

In [ ]:
np.meshgrid(r,c)